# Analyze results
Aug 28, 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

In [2]:
! realpath ../1_main_code/post_analysis_pandas.py

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/1_main_code/post_analysis_pandas.py


In [3]:
from ipywidgets import *

In [4]:
%matplotlib widget

In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *


In [6]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/1_main_code/')
import post_analysis_pandas as post


In [7]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [8]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [9]:
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [10]:
# parent_dir=u.result
parent_dir=dict1['128']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20201002_073628', '20201029_180105_no_adv_loss', '20…

In [11]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201112_074401_full_loss_b64_lr0.002_run2


In [12]:
# ### Read loss data
# with open (result_dir+'/metrics.pickle','rb') as f:
#     metrics_dict=pickle.load(f)
    
# df_metrics=pd.DataFrame([])
# for key in metrics_dict.keys(): df_metrics[key]=metrics_dict[key]
# df_metrics['step']=np.arange(df_metrics.shape[0])
# # df_metrics=df_metrics.dropna()

## Plot Losses

In [13]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)

In [14]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
46866,46866.0,14.0,0.226807,0.569993,0.796800,26.472057,75.889542,49.417480,-0.922249,49.361000,-0.851608,20.553944,-13.385874,-26.472057,0.143159
46867,46867.0,14.0,0.349257,0.275351,0.624609,16.293976,66.089691,49.795715,-0.810612,50.247612,-1.107796,15.026374,-12.419106,-16.293913,0.142805
46868,46868.0,14.0,0.487755,0.326111,0.813865,11.737450,60.919353,49.181904,-0.794124,49.390713,-0.592161,12.475077,-8.703245,-11.730945,0.146698
46869,46869.0,14.0,0.355177,3.111285,3.466462,33.543777,82.525658,48.981880,-1.061725,49.374016,-1.045859,12.571789,0.168033,-33.543777,0.145407
46870,46870.0,14.0,9.542706,0.292972,9.835677,0.283700,49.378189,49.094490,-0.896156,50.181763,-0.745410,-9.257021,-21.571545,7.247114,0.145635
46871,46871.0,14.0,0.531753,10.117256,10.649010,47.763802,96.181618,48.417816,-0.715934,48.938328,-0.342211,27.289829,10.115038,-47.763802,0.142644
46872,46872.0,14.0,6.922172,0.904505,7.826677,34.947552,87.117996,52.170448,-1.345004,49.299580,-1.622831,-4.694958,-41.703861,-34.947552,0.144594
46873,46873.0,14.0,0.642032,0.488552,1.130584,17.073959,65.781738,48.707775,-0.871725,49.023514,-1.060233,13.713549,-25.180182,-17.073895,0.143520
46874,46874.0,14.0,0.504185,0.694478,1.198663,17.539623,66.623825,49.084206,-1.352408,49.461655,-1.024691,27.843277,-5.361833,-17.539444,0.146083
46875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.439953,-1.286523,NaN,NaN,NaN,NaN


In [15]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [16]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-1.9427276706695555


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
18825,18825.0,6.0,2.580086,0.345132,2.925218,7.023273,53.850285,46.827011,-2.257545,46.315968,-2.483540,3.290156,-18.661156,-5.932309,0.139575
19082,19082.0,6.0,0.373535,2.071446,2.444980,17.413925,63.066978,45.653053,-2.016743,45.533264,-2.458509,24.737473,-6.645544,-17.315708,0.131330
18829,18829.0,6.0,1.030933,0.246922,1.277855,13.758848,58.856667,45.097816,-2.286974,45.680527,-2.366315,7.128915,-19.754635,-13.746202,0.133109
25405,25405.0,8.0,0.715105,0.228653,0.943758,25.883228,69.703499,43.820271,-2.284291,44.065430,-2.355333,19.978992,-24.209801,-25.883207,0.126933
19081,19081.0,6.0,8.043375,0.646584,8.689959,13.343850,58.627861,45.284012,-2.278835,45.259804,-2.351149,-6.245341,-37.631336,-11.329502,0.130191
18780,18780.0,6.0,2.313838,0.195121,2.508959,11.357116,56.380844,45.023727,-1.513350,44.825401,-2.346448,4.513052,-25.025677,-11.195529,0.132825
19071,19071.0,6.0,4.429436,0.625151,5.054587,7.405276,55.023232,47.617954,-2.154122,46.684914,-2.324779,1.532702,-23.627304,-4.859426,0.130700
44222,44222.0,14.0,0.244136,0.671707,0.915844,15.498057,61.079769,45.581711,-2.188246,43.110291,-2.310005,21.748592,-5.502801,-15.498035,0.133978
19086,19086.0,6.0,0.339171,2.305731,2.644902,33.321953,77.028336,43.706383,-2.089589,44.632713,-2.298663,24.379131,-2.373917,-33.321953,0.132645
13829,13829.0,4.0,1.710895,0.456976,2.167871,27.767206,72.655434,44.888229,-1.289879,43.494843,-2.263909,9.183650,-25.842411,-27.742561,0.135217


In [17]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
18825,18825.0,6.0,2.580086,0.345132,2.925218,7.023273,53.850285,46.827011,-2.257545,46.315968,-2.483540,3.290156,-18.661156,-5.932309,0.139575
19082,19082.0,6.0,0.373535,2.071446,2.444980,17.413925,63.066978,45.653053,-2.016743,45.533264,-2.458509,24.737473,-6.645544,-17.315708,0.131330
18829,18829.0,6.0,1.030933,0.246922,1.277855,13.758848,58.856667,45.097816,-2.286974,45.680527,-2.366315,7.128915,-19.754635,-13.746202,0.133109
25405,25405.0,8.0,0.715105,0.228653,0.943758,25.883228,69.703499,43.820271,-2.284291,44.065430,-2.355333,19.978992,-24.209801,-25.883207,0.126933
19081,19081.0,6.0,8.043375,0.646584,8.689959,13.343850,58.627861,45.284012,-2.278835,45.259804,-2.351149,-6.245341,-37.631336,-11.329502,0.130191
18780,18780.0,6.0,2.313838,0.195121,2.508959,11.357116,56.380844,45.023727,-1.513350,44.825401,-2.346448,4.513052,-25.025677,-11.195529,0.132825
19071,19071.0,6.0,4.429436,0.625151,5.054587,7.405276,55.023232,47.617954,-2.154122,46.684914,-2.324779,1.532702,-23.627304,-4.859426,0.130700
44222,44222.0,14.0,0.244136,0.671707,0.915844,15.498057,61.079769,45.581711,-2.188246,43.110291,-2.310005,21.748592,-5.502801,-15.498035,0.133978


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
25638,25638.0,8.0,0.399278,4.401143,4.800421,47.883598,90.507164,42.623566,-1.598955,40.303261,-1.812498,31.755106,2.818227,-47.883598,0.135206
18355,18355.0,5.0,0.397012,0.667667,1.064679,11.178524,55.280319,44.101795,-0.304477,40.447418,-0.500997,20.550323,-16.848412,-10.766525,0.134147
14284,14284.0,4.0,4.188451,0.435187,4.623638,34.182060,77.297523,43.115463,-0.488279,40.551979,-0.704100,0.474977,-41.200291,-34.181915,0.133227
33944,33944.0,10.0,0.172271,0.910535,1.082806,25.524643,70.325165,44.800522,-1.068654,40.649105,-1.655338,20.158842,-5.201316,-25.524643,0.132808
37333,37333.0,11.0,0.666942,0.321557,0.988499,18.705944,63.131950,44.426006,-0.044981,40.881599,-0.447029,16.745947,-16.140049,-18.705099,0.144605
25651,25651.0,8.0,0.627213,0.470339,1.097552,53.898712,95.452286,41.553574,-1.030858,40.988327,-1.117695,13.452620,-37.923004,-53.898712,0.133201
15544,15544.0,4.0,0.443521,3.084841,3.528362,47.310829,89.272263,41.961433,0.059099,41.133263,0.261106,23.411575,1.767545,-47.310829,0.136802
19158,19158.0,6.0,0.522368,0.338906,0.861274,12.698513,55.744381,43.045868,-0.765929,41.403175,-1.940412,23.963995,-22.802118,-12.380106,0.141024


In [18]:
df_metrics[df_metrics.step.isin([44861,44862,44863])]

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
44861,44861.0,14.0,0.492711,0.172961,0.665672,32.659344,77.798721,45.139378,-0.969819,44.394585,-0.699781,21.355446,-20.019323,-32.659344,0.134836
44862,44862.0,14.0,0.858162,0.408943,1.267105,25.583731,71.264153,45.680420,-0.669188,45.469311,-0.322350,12.642799,-24.767281,-25.583725,0.132501
44863,44863.0,14.0,0.666248,0.511266,1.177514,18.646225,65.019318,46.373093,-1.039029,45.867897,-0.712965,14.560790,-11.950204,-18.646221,0.133187


## Read stored chi-squares for images

In [23]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)

chi1=df.quantile(q=0.2,axis=0)['chi_1']
chi_spec=df.quantile(q=0.2,axis=0)['chi_spec3']
col_list=['epoch','step','chi_1','chi_spec1','chi_spec3']
df_sliced=df[(df.chi_spec1<chi_spec)&(df.chi_1<chi1)]


df_sliced.sort_values(by=['chi_spec3','chi_1'])[col_list].head(5)


,epoch,step,chi_1,chi_spec1,chi_spec3
25639,8,25638,0.053016,2.588209,20.905224
18824,6,18825,0.078294,3.803929,23.836920
38650,12,38649,0.081015,1.281339,24.925945
42278,13,42277,0.066225,0.576743,25.036353
40801,13,40800,0.101698,0.811031,25.235496


In [24]:
# df[(df.chi_spec1<chi_spec)&(df.chi_1<chi1)].sort_values(by=['chi_1','chi_spec3']).head(5)

In [25]:
# display(df.sort_values(by=['chi_1','chi_spec3']).head(5))
# df.sort_values(by=['chi_spec3','chi_1']).head(5)

In [38]:
best_idx=[]
best_idx.append(df_sliced.sort_values(by=['chi_1','chi_spec3']).head(4).index)
best_idx.append(df_sliced.sort_values(by=['chi_spec3','chi_1']).head(2).index)

best_idx=[i for j in best_idx for i in j]
print(best_idx)

[42502, 41747, 37812, 28554, 25639, 18824]


In [39]:
# best_idx=[18825,18829]
# df_metrics[df_metrics.step.isin(best_idx)]


In [40]:
df_best=df.loc[best_idx]
df_best['label']=df_best.epoch.astype(str)+'_'+df_best.step.astype(str)
print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

[(13, 42501), (13, 41746), (12, 37811), (9, 28553), (8, 25638), (6, 18825)]


In [41]:
# df_metrics.sort_values(by=['hist_chi']).head(8)
df_best[col_list]

,epoch,step,chi_1,chi_spec1,chi_spec3
42502,13,42501,0.013825,1.418745,28.207512
41747,13,41746,0.014823,1.426570,28.410875
37812,12,37811,0.016276,1.881051,26.904143
28554,9,28553,0.017946,1.947844,28.949121
25639,8,25638,0.053016,2.588209,20.905224
18824,6,18825,0.078294,3.803929,23.836920


In [42]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
18825,18825.0,6.0,2.580086,0.345132,2.925218,7.023273,53.850285,46.827011,-2.257545,46.315968,-2.483540,3.290156,-18.661156,-5.932309,0.139575
19082,19082.0,6.0,0.373535,2.071446,2.444980,17.413925,63.066978,45.653053,-2.016743,45.533264,-2.458509,24.737473,-6.645544,-17.315708,0.131330
18829,18829.0,6.0,1.030933,0.246922,1.277855,13.758848,58.856667,45.097816,-2.286974,45.680527,-2.366315,7.128915,-19.754635,-13.746202,0.133109
25405,25405.0,8.0,0.715105,0.228653,0.943758,25.883228,69.703499,43.820271,-2.284291,44.065430,-2.355333,19.978992,-24.209801,-25.883207,0.126933
19081,19081.0,6.0,8.043375,0.646584,8.689959,13.343850,58.627861,45.284012,-2.278835,45.259804,-2.351149,-6.245341,-37.631336,-11.329502,0.130191
18780,18780.0,6.0,2.313838,0.195121,2.508959,11.357116,56.380844,45.023727,-1.513350,44.825401,-2.346448,4.513052,-25.025677,-11.195529,0.132825
19071,19071.0,6.0,4.429436,0.625151,5.054587,7.405276,55.023232,47.617954,-2.154122,46.684914,-2.324779,1.532702,-23.627304,-4.859426,0.130700
44222,44222.0,14.0,0.244136,0.671707,0.915844,15.498057,61.079769,45.581711,-2.188246,43.110291,-2.310005,21.748592,-5.502801,-15.498035,0.133978


## Plots

In [43]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(fname,mmap_mode='r')[-4000:][:,0,:,:]

print(s_val.shape)

### Compute histogram and spectrum of raw data 
dict_val=post.f_compute_hist_spect(s_val,bins)
del(s_val)

(4000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:88: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [44]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec'],"Invalid mode %s"%(plot_type)

    fig=plt.figure(figsize=(6,6))
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_err
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    
#     plt.xscale('symlog',linthreshx=30)

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')

In [45]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec']))

interactive(children=(SelectMultiple(description='labels_list', options=('13_42501', '13_41746', '12_37811', '…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [58]:
step=best_idx[4]
fname='*gen_img_epoch-*_step-{0}.npy'.format(step)
# fname='gen_img_epoch-*_step-{0}.npy'.format(step)
fle=glob.glob(result_dir+'/images/'+fname)[0]
fle
imgs=np.load(fle)
print(imgs.shape)

(64, 128, 128)


In [59]:
f_plot_grid(imgs[10:28],cols=6,fig_size=(6,3))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(ip_fname,mmap_mode='r')[:1000,0,:,:]
print(s_val.shape)
f_plot_grid(s_val[100:118],cols=6,fig_size=(6,3))



(1000, 128, 128)
3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
df_metrics[df_metrics.step<2000]

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
0,0.0,0.0,155.384521,8994.041016,9149.425781,0.000000,51.541061,51.541061,2.091570,NaN,NaN,10250.341797,9178.380859,9016.666016,16.879873
1,1.0,0.0,210.029572,9208.751953,9418.781250,0.000000,48.151733,48.151733,1.631253,NaN,1.847819,10106.044922,9286.199219,8858.256836,0.181216
2,2.0,0.0,140.095917,8415.458984,8555.554688,0.000000,48.516342,48.516342,1.573048,111.063576,1.579643,9991.994141,8504.095703,8081.021973,0.123487
3,3.0,0.0,154.824448,8224.609375,8379.433594,0.000000,47.992279,47.992279,1.523583,47.979710,1.483361,9682.549805,8350.921875,8254.554688,0.124718
4,4.0,0.0,158.193985,8304.888672,8463.083008,0.000000,48.205635,48.205635,1.423072,48.113892,1.665658,9793.376953,8391.966797,7894.611816,0.125843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995.0,0.0,2.217350,1.219375,3.436724,11.998444,57.610607,45.612164,1.052104,44.142818,1.153962,8.724133,-10.659204,-10.684912,0.133720
1996,1996.0,0.0,1.678530,3.362064,5.040594,26.196583,71.000214,44.803631,1.019375,46.053162,1.230797,11.737940,-3.399637,-25.885695,0.132302
1997,1997.0,0.0,6.877900,1.407103,8.285003,4.220017,50.942043,46.722027,0.697115,45.434776,0.816212,-3.563159,-21.153328,3.383022,0.132426
1998,1998.0,0.0,0.078856,8.387987,8.466844,34.355717,78.574066,44.218346,0.909917,46.471924,0.440081,25.421879,5.292831,-34.355713,0.133718
